# Analysis of variance (ANOVA)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
kashti=sns.load_dataset('titanic')

In [ ]:
kashti.head(5)

In [ ]:
sns.boxplot(x='sex',y='age',data=kashti)

In [ ]:
sns.boxplot(x='class',y='age',data=kashti)

In [ ]:
phool=sns.load_dataset('iris')

In [ ]:
phool.head(5)

In [ ]:
phool.columns

In [ ]:
phool.describe()

In [ ]:
sns.boxplot(x='species',y='sepal_length',data=phool)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# One way ANOVA
mod = ols('sepal_length ~ species', data=phool).fit()


In [ ]:
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

# Pairwise comparison

In [ ]:
pair_t=mod.t_test_pairwise('species', method='bonferroni')
pair_t.result_frame

# Tukey HSD test

In [ ]:

pip install pingouin


In [ ]:
import pingouin as pg
# First develop anova table
aov=pg.anova(dv='sepal_length', between='species', data=phool, detailed=True)
print(aov)

In [ ]:
# Tukey HSD test
pt=pg.pairwise_tukey(dv='sepal_length', between='species', data=phool)
print(pt)

# Manova

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.multivariate.manova import MANOVA

In [ ]:
df=sns.load_dataset('iris')

In [ ]:
df.head(5)

In [ ]:
Manova=MANOVA.from_formula('sepal_length + sepal_width + petal_length + petal_width ~ species', data=df)

In [ ]:
print(Manova.mv_test())